In [1]:
import os

from matplotlib.pyplot import text
from dataloader.waterbird import Waterbird, WB_DomainTest
import clip
import torch
import clipfolder.clip as clipours
import torch.nn as nn
import argparse
import time
from PIL import Image
from skimage.transform import resize 


import random
import numpy as np

#set up the device as 2nd GPU
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")


In [2]:
model_list = []
for which_layer in [0,12]:#range(0, 13):
    model, preprocess = clipours.load('ViT-B/32', device, jit=False, extract_last_k_th_token=12 - which_layer) # model are ranked from the early layers to last layers interpretation.
    model_list.append(model)








transofmer total layers 12
 mask for layer 0
 mask for layer 1
 mask for layer 2
 mask for layer 3
 mask for layer 4
 mask for layer 5
 mask for layer 6
 mask for layer 7
 mask for layer 8
 mask for layer 9
 mask for layer 10
 mask for layer 11





transofmer total layers 12





transofmer total layers 12





transofmer total layers 12


In [3]:
no_attention_attn_blocks = [i for i in model_list[0].visual.transformer.resblocks.children()]
attn_blocks = [i for i in model_list[-1].visual.transformer.resblocks.children()]
logit_scale = model_list[-1].logit_scale.exp()

In [4]:
import json
from visual_genome import api
import requests
from PIL import Image as PIL_Image
from io import BytesIO
import matplotlib.pyplot as plt

with open('../mic/vaw_dataset/data/val.json') as f:
    annotations = json.load(f)

from tqdm import tqdm
import random
text = {""}
for i in tqdm(range(len(annotations))):
    text = text.union(annotations[i]['positive_attributes'])
    text = text.union(annotations[i]['negative_attributes'])
    # text = text.union([annotations[i]['object_name']])
text = list(text)


agg_dict = {}

skip_return = [None, None, None, None, None, None, None, None, None, None, None, None, None, None]

for annotation in annotations:
    try:
        polygon_arr = np.array(annotation['instance_polygon'])
        min_x = int(polygon_arr[0, :, 0].min())
        max_x = int(polygon_arr[0, :, 0].max())
        min_y = int(polygon_arr[0, :, 1].min())
        max_y = int(polygon_arr[0, :, 1].max())
    except:
        continue

    if annotation['image_id'] not in agg_dict:
        agg_dict[annotation['image_id']] = {"positive_attributes": [], "object_names": [], "bboxes": []}

    agg_dict[annotation['image_id']]['positive_attributes'] = list(set(agg_dict[annotation['image_id']]['positive_attributes']).union(annotation["positive_attributes"]))
    agg_dict[annotation['image_id']]['object_names'] = list(set(agg_dict[annotation['image_id']]['object_names']).union([annotation["object_name"]]))
    agg_dict[annotation['image_id']]['bboxes'].append([min_x, min_y, max_x, max_y])
agg_dict.pop("2364650", None)
img_ids = [i for i in agg_dict.keys()]

100%|██████████| 12286/12286 [00:00<00:00, 55758.43it/s]
/tmp/ipykernel_33395/2628356709.py:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  polygon_arr = np.array(annotation['instance_polygon'])


In [5]:
def get_interpretation_from_layer(x, layer, attn_blocks, ln_post, text_features, logit_scale, ans_with_cat, smooth_n = 500, center_l2s = None, get_top = True):
    with torch.no_grad():
        if layer <= len(attn_blocks) - 1:
            x = attn_blocks[layer](x)

        for i in range(layer+1,len(attn_blocks)):
            random_xs = []
            if smooth_n >= 1:
                for random_idx in range(smooth_n):
                    epsilon = torch.randn_like(x) * center_l2s[i-1]
                    x_random = x + epsilon / 20
                    x_random = attn_blocks[i](x_random)
                    random_xs.append(x_random)
                x = torch.stack(random_xs, dim=0).mean(dim=0)
            else:
                x = attn_blocks[i](x)
        
        random_xs = []
        if smooth_n >= 1 and layer <= len(attn_blocks) - 1:
            for random_idx in range(smooth_n):
                epsilon = torch.randn_like(x) * center_l2s[-1]
                x_random = x + epsilon
                x_random = x_random.permute(1, 0, 2)
                x_random = torch.cat([ln_post(x_random[:, idx, :]).unsqueeze(1) for idx in range(x_random.size(1))], dim=1)
                image_features = x_random @ model.visual.proj
                random_xs.append(image_features)
            image_features = torch.stack(random_xs, dim=0).mean(dim=0)
        else:
            image_features = x.permute(1, 0, 2)
            image_features = torch.cat([ln_post(image_features[:, idx, :]).unsqueeze(1) for idx in range(image_features.size(1))], dim=1)
            image_features = image_features @ model.visual.proj

        image_features = image_features / image_features.norm(dim=1, keepdim=True)


        logits_img = (logit_scale * image_features) @ text_features.t()
        
        if get_top:
            pseudo_target = np.argmax(logits_img.cpu().numpy(), axis=2)
            ans_2 = []
            for bs in range(pseudo_target.shape[0]):
                ans_3 = []
                for it in range(0, pseudo_target.shape[1]): # for each token, put 1 here to avoid CLS token replace
                    index = pseudo_target[bs, it]
                    ans_3.append(ans_with_cat[index])
                ans_2.append(ans_3)

            return ans_2
        else:
            ans_with_cat = np.array(ans_with_cat)
            psuedo_target = np.argsort(logits_img.cpu().numpy(), axis=2)
            psuedo_target = psuedo_target[:, :, ::-1]
            ans_2 = []
            for bs in range(psuedo_target.shape[0]):
                ans_3 = []
                for it in range(0, psuedo_target.shape[1]):
                    index = psuedo_target[bs, it]
                    ans_3.append(ans_with_cat[index])
                ans_2.append(ans_3)
            
            return ans_2
        
def get_interpretation_all_layers(x, attn_blocks, no_attention_attn_blocks, ln_post, text_features, logit_scale, ans_with_cat, smooth_n = 500, center_l2s = None, get_top = True):
    ans = []
    with torch.no_grad():
        x = model.visual.conv1(x.type(torch.cuda.HalfTensor))
        x = x.reshape(x.shape[0], x.shape[1], -1)  # shape = [*, width, grid ** 2]   [B, channel, H*W]
        x = x.permute(0, 2, 1)  # shape = [*, grid ** 2, width]       [B, H*W, channel]
        x = torch.cat([model.visual.class_embedding.to(x.dtype) + torch.zeros(x.shape[0], 1, x.shape[-1], dtype=x.dtype, device=x.device), x], dim=1)  # shape = [*, grid ** 2 + 1, width]
        x = x + model.visual.positional_embedding.to(x.dtype)
        x = model.visual.ln_pre(x)
        x = x.permute(1, 0, 2)  # NLD -> LND    [H*W, B, channel]

        for i in range(len(attn_blocks)):
            interpretations = get_interpretation_from_layer(x, i, no_attention_attn_blocks, ln_post, text_features, logit_scale, ans_with_cat, smooth_n = smooth_n, center_l2s = center_l2s, get_top = get_top)
            ans.append(interpretations)
            x = attn_blocks[i](x)
        interpretations = get_interpretation_from_layer(x, i+1, no_attention_attn_blocks, ln_post, text_features, logit_scale, ans_with_cat, smooth_n = smooth_n, center_l2s = center_l2s, get_top = get_top)
        ans.append(interpretations)
        
        return ans


In [6]:
import torch
from torch.utils.data import Dataset
import torchvision.transforms as transforms
class VAWDataset(Dataset):
    def __init__(self, img_ids, root_dir, 
                 transform=None):
        self.transform = transform
        self.root_dir = root_dir
        self.img_ids = img_ids

    def __len__(self):
        return len(self.img_ids)

    def __getitem__(self, idx):
        img_id = self.img_ids[idx]
        img = Image.open(self.root_dir + '/' + img_id + '.jpg').convert("RGB") # you can use PIL to open the image

        if self.transform:
            img = self.transform(img)

        return img
    
train_dataset = VAWDataset(list(agg_dict.keys()), root_dir='vaw_images', transform=preprocess)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 256, shuffle=True, num_workers=8)


In [7]:
import torch
from torch.utils.data import Dataset
import torchvision.transforms as transforms
import multiprocessing as mp

import multiprocessing
import numpy as np
from PIL import Image
from skimage.transform import resize


class VAWVerificationDataset(Dataset):
    def __init__(self, img_ids, root_dir, 
                 transform=None, random_n = 5):
        self.transform = transform
        self.root_dir = root_dir
        self.img_ids = img_ids
        self.random_n = random_n

    def __len__(self):
        return len(self.img_ids)

    def __getitem__(self, idx):
        img_id = self.img_ids[idx]
        img = Image.open(self.root_dir + '/' + img_id + '.jpg').convert("RGB") # you can use PIL to open the image
        images = []

        original_image = np.array(img)
        original_image = Image.fromarray(original_image)
        images.append(original_image)
        masked_image = np.array(img)
        for bbox in agg_dict[img_id]['bboxes']:
            min_x, min_y, max_x, max_y = bbox
            try:
                masked_image[min_y:max_y, min_x:max_x, :] = 0
            except:
                print("error", img_id)
        
        masked_image = Image.fromarray(masked_image)
        images.append(masked_image)

        for i in range(self.random_n):
            masked_image = np.array(img)

            total_box_area = 0
            for bbox in agg_dict[img_id]["bboxes"]:
                min_x, min_y, max_x, max_y = bbox
                box_width = max_x - min_x
                box_height = max_y - min_y
                box_area = box_width * box_height
                total_box_area += box_area
                total_box_area_ratio = total_box_area / (masked_image.shape[0] * masked_image.shape[1])

                mask = np.random.rand(int(masked_image.shape[0]/50), int(masked_image.shape[1]/50))
                mask = mask < (total_box_area_ratio / (1-total_box_area_ratio))
                mask = resize(mask,masked_image.shape)

            for bbox in agg_dict[img_id]["bboxes"]:
                min_x, min_y, max_x, max_y = bbox
                mask[min_y:max_y, min_x:max_x, :] = False

            masked_image[mask] = 0
            
            masked_image = Image.fromarray(masked_image)
            images.append(masked_image)        
        
        if self.transform:
            images = [self.transform(image) for image in images ]
        
        object_names = agg_dict[img_id]["object_names"]

        # print(object_names)
        return ",".join(object_names), *images
    
verification_dataset = VAWVerificationDataset(list(agg_dict.keys()), random_n = 2, root_dir='vaw_images', transform=preprocess)
verification_loader = torch.utils.data.DataLoader(verification_dataset, batch_size = 40, shuffle=False, num_workers=mp.cpu_count())


In [8]:
center_l2s_batches = []
with_attention_center_batches = []
no_attention_center_batches = []

with torch.no_grad():
    for i, batch in enumerate(train_loader):
        center_l2s = []
        with_attention_center_layers = []
        no_attention_center_layers = []
        X = batch
        X = X.to(device)

        with torch.no_grad():
            x = model.visual.conv1(X.type(torch.cuda.HalfTensor))
            x = x.reshape(x.shape[0], x.shape[1], -1)  # shape = [*, width, grid ** 2]   [B, channel, H*W]
            x = x.permute(0, 2, 1)  # shape = [*, grid ** 2, width]       [B, H*W, channel]
            x = torch.cat([model.visual.class_embedding.to(x.dtype) + torch.zeros(x.shape[0], 1, x.shape[-1], dtype=x.dtype, device=x.device), x], dim=1)  # shape = [*, grid ** 2 + 1, width]
            x = x + model.visual.positional_embedding.to(x.dtype)
            x = model.visual.ln_pre(x)
            x = x.permute(1, 0, 2)  # NLD -> LND    [H*W, B, channel]

            for i in range(len(attn_blocks)):
                x = attn_blocks[i](x)
                x_no_attention = no_attention_attn_blocks[i](x)
                with_attention_center = x.mean(axis=1)
                no_attention_center = x_no_attention.mean(axis=1)
                with_attention_center_layers.append(with_attention_center)
                no_attention_center_layers.append(no_attention_center)
        with_attention_center_batches.append(with_attention_center_layers)
        no_attention_center_batches.append(no_attention_center_layers)

center_l2s = []
for layer in range(12):
    with_attention_center_this_layer = [with_attention_center_batches[i][layer].unsqueeze(1) for i in range(len(with_attention_center_batches))]
    with_attention_center_this_layer = torch.cat(with_attention_center_this_layer, dim=1)
    with_attention_center = with_attention_center_this_layer.mean(dim=1)
    no_attention_center_this_layer = [no_attention_center_batches[i][layer].unsqueeze(1) for i in range(len(no_attention_center_batches))]
    no_attention_center_this_layer = torch.cat(no_attention_center_this_layer, dim=1)    
    no_attention_center = no_attention_center_this_layer.mean(dim=1)

    center_l2 = torch.sqrt((with_attention_center - no_attention_center)**2).unsqueeze(1)    
    center_l2s.append(center_l2)
    

In [31]:
with torch.no_grad():
    rank_change_dataset = []
    for batch in tqdm(verification_loader):
        object_names_raw = batch[0]
        object_names = []
        for object_name_raw in object_names_raw:
            object_names += object_name_raw.split(",")
        
        all_text = text + object_names
        ans_with_cat = all_text
        all_prompts = [f"this can be described as " + each for each in all_text]
        text_tokens = clip.tokenize(all_prompts).to(device)

        with torch.no_grad():
            image_fea, text_fea = model_list[-1](batch[1].to(device), text_tokens, get_all_last=True)
            text_fea = text_fea / text_fea.norm(dim=-1, keepdim=True)
        
        interpretations_all_type = []
        for image_type_idx in range(1, len(batch)):
            # type_interpretations = get_interpretation_all_layers(batch[image_type_idx].to(device), attn_blocks, no_attention_attn_blocks, model_list[-1].visual.ln_post, text_fea, logit_scale, ans_with_cat, smooth_n = 0, center_l2s = None, get_top = False)
            type_interpretations = get_interpretation_all_layers(batch[image_type_idx].to(device), attn_blocks, no_attention_attn_blocks, model_list[-1].visual.ln_post, text_fea, logit_scale, ans_with_cat, smooth_n = 100, center_l2s = center_l2s, get_top = False)
            interpretations_all_type.append(type_interpretations)

        
        for image_i in range(len(interpretations_all_type[0][0])):
            original_top_interpretations = [[interpretations_all_type[0][i][image_i][j][0] for j in range(50)] for i in range(13)]
            rank_change_image = []
            for image_type_i in range(1,len(interpretations_all_type)):
                rank_change_image_type = []
                for i in range(13):
                    rank_change_layer = []
                    for j in range(50):
                        rank_change_layer.append(np.where(interpretations_all_type[image_type_i][i][image_i][j] == original_top_interpretations[i][j])[0][0])
                    rank_change_image_type.append(rank_change_layer)
                rank_change_image.append(rank_change_image_type)        
            rank_change_dataset.append(rank_change_image)

100%|██████████| 83/83 [1:02:00<00:00, 44.82s/it]


In [84]:
rank_change_dataset = np.array(rank_change_dataset)

In [10]:
center_l2s_arr = torch.cat(center_l2s, dim = 1).cpu().numpy()
center_l2s_arr.shape

(50, 12, 768)

In [11]:
print("VAW")
print(f"CLS drift {center_l2s_arr.mean(axis = 1).mean(axis = 1)[0]}")
print(f"other tokens mean drift {center_l2s_arr.mean(axis = 1).mean(axis = 1)[1:].mean()}")

VAW
CLS drift 0.0791015625
other tokens mean drift 0.0870361328125


In [14]:
center_l2s_arr.mean(axis = 0).mean(axis = 1)

array([0.2732 , 0.0604 , 0.05307, 0.04626, 0.03717, 0.0409 , 0.03064,
       0.0369 , 0.04086, 0.0758 , 0.1335 , 0.2135 ], dtype=float16)

In [ ]:
print(f"overall vaw mask rank change {rank_change_dataset[:,0,:,:].mean()}, random mask rank change {rank_change_dataset[:,1:,:,:].mean()}")

for layer_idx in range(13):
    print(f"vaw mask rank change layer {layer_idx} {rank_change_dataset[:,0,layer_idx,:].mean()}, random mask rank change layer {layer_idx} {rank_change_dataset[:,1:,layer_idx,:].mean()}")